In [1]:
import re
import requests
import json
import os
import time
import traceback
from multiprocessing import Pool

In [2]:
def open_text(i,tickCount,log):
    try :
        with open ('D:\\新增資料夾\\{}\\url_591_{}.txt'.format(i,tickCount),'r',encoding='utf8') as f:
            text = f.read()
            text = text.replace('\\n','').replace('&nbsp;','').replace('\\r','').replace('<br>','')
            text = text.replace('<br />','').replace(' ','').replace('\\u3000','')
    except Exception as e :
        text = ''
        print('url_591_{} with exception {} at open_text'.format(tickCount,e))
        log.append('url_591_{} with exception {} at open_text , '.format(tickCount,e))
        pass
    return text

In [3]:
def get_url(text,tickCount,log):
    try:
        url =str(*re.findall('\'url\':\'(.*)\',\'html\':',text))
    except Exception as e :
        print('url_591_{} with exception {} at get_url'.format(tickCount,e))
        log.append('url_591_{} with exception {} at open_text , '.format(tickCount,e))
        pass
    return url

In [4]:
def get_title(text,tickCount,log):
    try:
        title =str(*re.findall('houseInfoTitle\">(.*)</span></h1><divc',text))
    except Exception as e :
        print('url_591_{} with exception {} at get_title'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_title , '.format(tickCount,e))
        pass
    return title

In [5]:
def get_address(text,tickCount,log):
    try:
        address = str(*re.findall('class=\"addr\">(.*)</span><istyle=\"f',text))
    except Exception as e :
        print('url_591_{} with exception {} at get_address'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_address , '.format(tickCount,e))
        pass
    return address

In [6]:
def get_landlord(text＿tickCount,tickCount,log):
    try:
        landlord = str(*re.findall('avatarRight">(.*)</div><divclass="infoTwo',text))
        landlord = landlord.split('<i>')[1].split('</i>')[0]
    except Exception as e :
        print('url_591_{} with exception {} at get_landlord'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_landlord , '.format(tickCount,e))
        pass
    return landlord

In [7]:
def get_stories(text,tickCount,log):
    try:
        stories = str(*re.findall('</li><li>樓層:(.*)</li><li>型態',text)).replace('F','').split('/')[1]
        stories = int(stories)
    except Exception as e :
        stories = 0
        print('url_591_{} with exception {} at get_stories'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_stories , '.format(tickCount,e))
        pass
    return stories

In [8]:
def get_floor(text,tickCount,log,stories):
    try:
        if re.findall('型態:(.*)</li><li>現況',text) == '透天厝':
            print('url_591_{} is 透天厝 '.format(tickCount))
        
        floor = str(*re.findall('</li><li>樓層:(.*)</li><li>型態',text)).replace('F','').split('/')[0]
#         floor = str(*re.findall('[0-9]',floor)[0])
        if floor == 'B1':
            floor = -1
        elif floor == 'B2':
            floor = -2
        elif floor == '頂樓加蓋':
            floor = stories
        floor = int(floor)
    except Exception as e :
        floor = 0
        print('url_591_{} with exception {} at get_floor'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_floor , '.format(tickCount,e))
        pass
    return floor 

In [9]:
def get_description(text,tickCount,log):
    try:
        description = str(*re.findall('"houseIntro"onselectstart="returnfalse;"style="-moz-user-select:none;user-select:none;">(.*)</div><!--仲',text))
        if ('>' in description) & ('<' in description):
            description = re.findall('[\u4E00-\u9FFF]+[0-9\u3000-\u303F-，,:]*',description)
            description = ''.join(description)
    except Exception as e :
        print('url_591_{} with exception {} at get_description'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_description , '.format(tickCount,e))
        pass
    return description       

In [10]:
def get_rent(text,tickCount,log):
    try:
        rent = str(*re.findall('priceclearfix\"><i>(.*)<b>元/月',text)).replace(',','')
        rent = int(rent)
    except Exception as e :
        rent = 0
        print('url_591_{} with exception {} at get_rent'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_rent , '.format(tickCount,e))
        pass
    return rent

In [11]:
def get_cityID(text,tickCount,log):
    try:
        cityID = str(*re.findall('北市</a>(.*)區</a>&gt;<ahref="https://rent.591',text)).split('">')[1]
        cityID = code_[cityID+'區']
        cityID = int(cityID)
    except Exception as e :
        cityID = 0
        print('url_591_{} with exception {} at get_cityID'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_cityID , '.format(tickCount,e))
        pass
    return cityID

In [12]:
def get_id_(text,tickCount,log):
    try:
        id_ = str(*re.findall('rent-detail-(.*).html',url))
    except Exception as e :
        print('url_591_{} with exception {} at get_id_'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_id_ , '.format(tickCount,e))
        pass
    return id_

In [13]:
def get_lat_lng(text,tickCount,log):
    try:
        res1 = requests.get('https://rent.591.com.tw/map-houseRound.html?type=1&post_id={}&detail=detail'.format(id_),headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36','cookie':'SID=9QQe2QfwhGx82ooP_JH5tu1yI-BxvwznWPVpNbrHBvmhWPpCfZY7zkaELbi6DfZhwEQRHg.; HSID=Afxnv5ZwuQwRWRWya; SSID=AlRtPJRJTbUcs6UDW; APISID=xYcDioiP2XVhShKT/AYK8j_bZb28ydDLlG; SAPISID=0Asl7o191762EgbJ/AEGTSHtaQ1IQAPzZp; OTZ=3980485_24_24__24_; NID=110=YgM5qNYSmLpllU4f81F3_TAxYfK4p-dePjdyOv8P04k_bCXQTppuCinvv8eR51Az7g48zXNEwpeoiBMtGhWvZ6osbHBb_YSdYl-jelOyeICH3cWi651mzNRN6vER7lvjLoejs6Yiy4_qnJtIeGOy6JDKEb4ghxluTHz4cJ8KyaH9zCZU8UVpPAs6Z37Jt3ZnbIrFqFq7cocX__9LH4viTBsJs83HTf4kt2V8; SIDCC=AA248bdQFRrfX8NJdHP01CCHu0uFPqSC61ZLorz0RcgnCqcARlzDRYZnCL1-7l-_nGe1omgvi8iPoKGBHnI'})
        lat = str(*re.findall('l=zh-TW&q=(.*)&z=17&ou',res1.text)).split(',')[0]
        lng = str(*re.findall('l=zh-TW&q=(.*)&z=17&ou',res1.text)).split(',')[1] 
        lat = float(lat)
        lng = float(lng)
    except Exception as e :
        lat = 0.0
        lng = 0.0
        print('url_591_{} with exception {} at get_lat_lng'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_lat_lng , '.format(tickCount,e))
        pass
    latlng = (lat,lng)
    return (latlng)

In [14]:
def get_sex(text,tickCount,log):
    try:
        sex = str(*re.findall('性別要求</div><divclass=\"two\"><span>：</span><emtitle=\"(.*)生</em></di',text)).split('">')[0]
        if sex == '女生' :
            sex = 'F'
        elif sex == '男生' :
            sex = 'M'
        else :
            sex = 'A'
    except Exception as e :
        print('url_591_{} with exception {} at get_sex'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_sex , '.format(tickCount,e))
        pass
    return sex

In [15]:
def get_space(text,tickCount,log):
    try:
        space = str(*re.findall('<li>坪數:(.*)</li><li>樓層',text)).split('坪')[0]
        space = str(re.findall('[0-9.]*',space)[0])
        space = float (space)
    except Exception as e :
        space = 0
        print('url_591_{} with exception {} at get_space'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_space , '.format(tickCount,e))
        pass
    return space

In [16]:
def get_pet(text,tickCount,log):
    try:
        pet = str(*re.findall('養寵物</div><divclass=\"two\"><span>：</span><emtitle=\"(.*)以</em></di',text)).split('">')[0]
        if pet == '可以' :
            pet = 'Y'
        else :
            pet = 'N'
    except Exception as e :
        print('url_591_{} with exception {} at get_pet'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_pet , '.format(tickCount,e))
        pass
    return pet

In [17]:
def get_cook(text,tickCount,log):
    try:
        cook = str(*re.findall('開伙</div><divclass=\"two\"><span>：</span><emtitle=\"(.*)以</em></di',text)).split('">')[0]
        if cook == '可以' :
            cook = 'Y'
        else :
            cook = 'N'
    except Exception as e :
        print('url_591_{} with exception {} at get_cook'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_cook , '.format(tickCount,e))
        pass
    return cook

In [18]:
def get_temp(text,tickCount,log):
    try:
        temp = str(*re.findall('最短租期</div><divclass=\"two\"><span>：</span><emtitle=\"(.*)年</em></di',text)).split('">')[0]
        if temp != '一年' :
            temp = 'Y'
        else :
            temp = 'N'
    except Exception as e :
        print('url_591_{} with exception {} at get_temp'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_temp , '.format(tickCount,e))
        pass
    return temp

In [19]:
#patternNa 
def get_pattern(text,tickCount,log):
    try:
        pattern = str(*re.findall('隔間材料</div><divclass=\"two\"><span>：</span><emtitle=\"(.*)</em></div><liclass=\"clear',text)).split('">')[0]
        if bool(pattern) == False:
            pattern = 'NA'
            #global patternNa += 1
    except Exception as e :
        print('url_591_{} with exception {} at get_pattern'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_pattern , '.format(tickCount,e))
        pass
    return pattern
    

In [20]:
def get_label(typ,tickCount,log):
    try:
        #label = str(*re.findall('區</a>&gt;<ahref=\"https://rent.591.com.tw/home/rent/index/r1s10k4.html\">(.*)</a>&gt;<ahref=\"https://',text)).split('</')[0]
        #if bool(label) == False:
            #print('url_591_{} didnt get label'.format(tickCount))
        if typ in  ['TPE_雅房','NEWTPE_雅房']:
            label = '雅'
        elif typ in ['TPE_整層住家','NEWTPE_整層住家']:
            label = '住'
        else :
            label = '套'      
    except Exception as e :
        print('url_591_{} with exception {} at get_label'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_label , '.format(tickCount,e))
        pass
    return label

In [21]:
def get_time(text,tickCount,log):
    try:
        time = str(*re.findall('time\':\'(.*)\'',text))
    except Exception as e :
        print('url_591_{} with exception {} at get_time'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_time , '.format(tickCount,e))
        pass
    return time

In [22]:
def get_imgsrc(text,tickCount,log):
    try:
        imgsrc = str(*re.findall('imgsrc=\"//statics.591.com.tw/tools/showPhone.php\?(.*)\"/></span></div><!--聯繫',text))
    except Exception as e :
        print('url_591_{} with exception {} at get_imgsrc'.format(tickCount,e))
        log.append('url_591_{} with exception {} at get_imgsrc , '.format(tickCount,e)) 
        pass
    return imgsrc

In [23]:
def make_code():
    code = '''中正區 100
    大同區 103
    中山區 104
    松山區 105
    大安區 106
    萬華區 108
    信義區 110
    士林區 111
    北投區 112
    內湖區 114
    南港區 115
    文山區 116
    萬里區 207
    金山區 208
    板橋區 220
    汐止區 221
    深坑區 222
    石碇區 223
    瑞芳區 224
    平溪區 226
    雙溪區 227
    貢寮區 228
    新店區 231
    坪林區 232
    烏來區 233
    永和區 234
    中和區 235
    土城區 236
    三峽區 237
    樹林區 238
    鶯歌區 239
    三重區 241
    新莊區 242
    泰山區 243
    林口區 244
    蘆洲區 247
    五股區 248
    八里區 249
    淡水區 251
    三芝區 252
    石門區 253'''
    code = code.replace('\n',',')
    code_={}
    for row in code.split(',') :
        code_[row.split()[0]]=row.split()[1]
    return code_


In [24]:
def make_json_(cityID,url,title,address,label,pattern,floor,stories,rent,latlng,sex,space,pet,cook,time,landlord,description,temp):
    json_ = {
     'cityID':cityID,
     'url':url,
     'title':title,
     'address':address,
     'pattern':pattern,
     'floor':floor,
     'stories':stories,
     'label':label,
     'lat':latlng[0],
     'lng':latlng[1],
     'rent':rent,
     'sex':sex,
     'space':space,
     'smoke':'N',
     'pet':pet,
     'cook':cook,
     'updateDate':time,
     'landlord':landlord,
     'description':description,
     'temp':temp,
    }
    return json_

In [25]:
def write_json_(json_,id_):
    with open('D:\\json_591\\{}.json'.format(id_),'w',encoding='utf8') as f:
            json.dump(json_,f)                                              

In [26]:
typ = ['TPE_獨立套房','TPE_整層住家','TPE_雅房','TPE_分租套房']
code_ = make_code()
img = {}
for i in typ:
    print(i)
    log =[]
    for tickCount in range(1,10000):
        try:
            text = open_text(i,tickCount,log)
            if bool(text) == False:
                break
            address = get_address(text,tickCount,log)
            cityID = get_cityID(text,tickCount,log)
            url = get_url(text,tickCount,log)
            id_ = get_id_(text,tickCount,log)
            title = get_title(text,tickCount,log)
            pattern = get_pattern(text,tickCount,log)
            stories = get_stories(text,tickCount,log)
            floor = get_floor(text,tickCount,log,stories)
            label = get_label(text,tickCount,log)
            rent = get_rent(text,tickCount,log)
            latlng = get_lat_lng(text,tickCount,log)
            sex = get_sex(text,tickCount,log)
            space = get_space(text,tickCount,log)
            pet = get_pet(text,tickCount,log)
            cook = get_cook(text,tickCount,log)
            time = get_time(text,tickCount,log)
            landlord = get_landlord(text,tickCount,log)
            description = get_description(text,tickCount,log)
            temp = get_temp(text,tickCount,log)
            json_ = make_json_(cityID,url,title,address,pattern,floor,stories,label,rent,latlng,sex,space,pet,cook,time,landlord,description,temp)
            write_json_(json_,id_)
            # phone img
            imgsrc = get_imgsrc(text,tickCount,log)
            img[id_]='https://statics.591.com.tw/tools/showPhone.php?'+imgsrc
        except Exception as e :
            print(e)
            pass
    with open('D:\\json_591\\log_errorLog_{}'.format(i),'w',encoding='utf8') as f:
            f.write(str(log))
with open('D:\\json_591\\log_imgsrc_{}','w',encoding='utf8') as f:
    f.write(str(img))

TPE_獨立套房
[Errno 2] No such file or directory: 'D:\\json_591\\5431826.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5466578.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5510278.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5493233.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5494448.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5489751.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5514479.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5510877.json'
[Errno 2] No such file or directory: 'D:\\json_591\\4370096.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5498950.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5498618.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5493230.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5467918.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5488130.json'
url_591_15 with exception ("bad handshake: Error([('SSL routines', 

From cffi callback <function _verify_callback at 0x00000000060A1F28>:
Traceback (most recent call last):
  File "D:\Users\Java\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 201, in wrapper
    @wraps(callback)
KeyboardInterrupt


[Errno 2] No such file or directory: 'D:\\json_591\\5444143.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5461131.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5488814.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5451726.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5512297.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5513108.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5508610.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5458492.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5513627.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5521633.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5498349.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5528888.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5520068.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5466471.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5462800.json'
[Errno 2] 

[Errno 2] No such file or directory: 'D:\\json_591\\5525690.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5485992.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5454331.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5456578.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5527936.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5504493.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5523160.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5511810.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5526255.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5439207.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5458293.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5465934.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5061493.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5472077.json'
[Errno 2] No such file or directory: 'D:\\json_591\\5442259.json'
[Errno 2] 

KeyboardInterrupt: 

In [ ]:
#  with open('D:\\json_591\\log_errorLog','r',encoding='utf8') as f:
#             tex = f.read()

In [ ]:
# log=  '''
# '''

In [ ]:
# log = log.replace('\n',',')
# log

In [ ]:
# text = tex.split('\n')

In [ ]:
# with open('D:\\json_591\\log_errorLog'.format(i),'w',encoding='utf8') as f:
            f.write(str(log))
            f.close

In [ ]:
# len(text)